# Домашнее задание 2.

- Влияние стандартизации признаков на модель линейной регрессии
- Интерпретация модели линейной регрессии 

В качестве набора данных мы будем пользоваться данными о стоимости домов в King County, Wahington, USA. Перед вами стоит задача предсказания стоимости дома.

## 1. Обучите линейную модель

1. Для обучения используйте только набор данных `kc_house_data_train.csv`, набором данных `kc_house_data_test.csv` вы воспользуетесь в самом конце для тестирования качества.

2. Разбейте выборку `kc_house_data_train.csv` на тренировочную и валидационную в отношении 7:3. Используйте `sklearn.model_selection.train_test_split`. Исключите из рассмотрения столбцы `id`, `zipcode`, `lat`, `long`.

3. Обучите линейную модель `sklearn.linear_model.LinearRegression` (с параметрами по умолчанию) на предсказание столбца `price`.  - Чему равно качество такой модели в терминах Mean Squared Error на тренировочной и валидационной выборке?
- Какие признаки имеют наибольший положительный и отрицательный вес? Насколько это согласуется с вашей интуицией?


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE

In [25]:
pd.options.display.max_columns=30

In [26]:
fd = pd.read_csv('kc_house_data_train.csv', index_col=0)
fd_test = pd.read_csv('kc_house_data_test.csv', index_col=0)

pd.read_csv('kc_house_data_train.csv', index_col=0).head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
21162,8691440440,20141003T000000,882990.0,4,3.50,3560,6562,2.0,0,0,3,10,3560,0,2014,0,98075,47.5929,-121.974,3710,6562
16877,2725069108,20140805T000000,750000.0,3,3.25,4610,81935,2.0,0,0,4,9,4610,0,1984,0,98074,47.6217,-122.021,2900,43500
16116,8026200080,20140715T000000,372000.0,4,1.75,1890,10550,1.0,0,0,5,7,1010,880,1969,0,98056,47.5147,-122.193,1930,7291


In [52]:
X = pd.DataFrame()
x_test = pd.DataFrame()

for i in fd:
    if(i != 'price' and i != 'lat' and i != 'long' and i != 'zipcode' and i != 'date' and i != 'id'):
        X[i] = fd[i]
        x_test[i] = fd_test[i]
        
print(X)
        
Y = fd['price']
y_test = fd_test['price']

x_train, x_valid, y_train, y_valid = train_test_split(X, Y, train_size=0.7, random_state=5)

model = LinearRegression()
model.fit(x_train, y_train)

y_valid_pred = model.predict(x_valid)
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)
   
for i in y_valid_pred:
    if i < 0:
       print(i) 

       bedrooms  bathrooms  sqft_living  sqft_lot  floors  waterfront  view  \
21162         4       3.50         3560      6562     2.0           0     0   
16877         3       3.25         4610     81935     2.0           0     0   
16116         4       1.75         1890     10550     1.0           0     0   
9557          2       1.00          840      5510     1.0           0     0   
16302         4       3.50         5550     28078     2.0           0     2   
...         ...        ...          ...       ...     ...         ...   ...   
20147         3       1.50         1420      1192     2.0           0     0   
8294          4       3.25         2885     33671     2.0           0     0   
10406         5       2.25         3260      7969     2.0           0     0   
6701          4       2.50         3100     10000     2.0           0     0   
14811         4       3.00         2570      4000     2.0           0     0   

       condition  grade  sqft_above  sqft_basement 

In [53]:
mean_squared_error(y_valid, y_valid_pred)

44600692064.984795

In [54]:
mean_squared_error(y_train, y_train_pred)

47752321019.68232

In [55]:
model = LinearRegression()
model.fit(x_train, y_train)
print(model.coef_)
abscoef = abs(model.coef_)
print(abscoef/abscoef.max())

[-3.93924030e+04  6.04054791e+04  1.06575446e+02 -7.80530689e-02
  1.83254462e+04  5.43770594e+05  4.51696252e+04  1.47774083e+04
  1.19568462e+05  5.33640781e+01  5.32113679e+01 -3.72072629e+03
  8.13790238e-01  2.24754267e+01 -4.32520474e-01]
[7.24430548e-02 1.11086329e-01 1.95993396e-04 1.43540437e-07
 3.37006937e-02 1.00000000e+00 8.30674290e-02 2.71758136e-02
 2.19887693e-01 9.81371163e-05 9.78562806e-05 6.84245586e-03
 1.49656904e-06 4.13325526e-05 7.95409827e-07]


## 2. Стандартизация признаков

Поскольку в линейной модели мы ищем целевую характеристику ввиде линейной комбинации признаков, то мы бы хотели чтобы величины признаков измерялись в сопоставимых масштабах. Сравните различные подходы к стандартизации признаков:

1. Стандартизация или z-scoring: (x - mean(x)) / std(x). Среднее каждого признака становится равно 0, стандартное отклонение становится равно 1.
2. Нормализация к [0, 1]: (x - min(x)) / (max(x) - min(x))
3. Логарифмирование: log(x + 1). Потенциальным недостатком первых двух подходов является необходимость подсчета статистик на тренировочной выборке и применение их на тестовой.

Обратите внимание что подсчет всех статистик mean, std, min, max должен производиться на тренировочной части данных и применяться к тестовой (потому что на практике у вас нет никакой статистики на этапе предсказание, вы делаете предсказание для конкретного 1ого дома).

Воспользуйтесь методами `sklearn.preprocessing.StandardScaler` и `sklearn.preprocessing.MinMaxScaler`. Обучайте модель линейной регрессии на тренировочной части данных, делайте предсказание и оценивайте качество на валидационной.

4. Какой из подходов дает наилучшее качество (включая отсутствие какой бы то нибыло стандартизации данных)?
5. Как меняются веса признаков в линейной модели при применении различных видов стандартизации?

In [56]:
zscoring = StandardScaler()
zscoring.fit(x_train)

x_train_new1 = zscoring.transform(x_train)
x_valid_new1 = zscoring.transform(x_valid)
x_test_new1 = zscoring.transform(x_test)

model = LinearRegression()
model.fit(x_train_new1, y_train)

y_valid_pred1 = model.predict(x_valid_new1)
y_test_pred1 = model.predict(x_test_new1)

print(mean_squared_error(y_valid, y_valid_pred1))

print(model.coef_)
abscoef = abs(model.coef_)
print(abscoef/abscoef.max())

44600692064.987686
[ -38461.61438468   46576.52400712   75015.83554764   -3143.20198512
    9800.43758878   45666.81032933   34645.06267487    9510.36158203
  139561.3802391    64778.4392752    34770.93468401 -109828.56407384
     322.10888184   15371.81990652  -11996.50414092]
[0.27558924 0.33373505 0.53751142 0.022522   0.07022313 0.32721667
 0.24824248 0.06814465 1.         0.46415734 0.24914439 0.78695527
 0.00230801 0.11014379 0.08595862]


In [57]:
minmax = MinMaxScaler()
minmax.fit(x_train)

x_train_new2 = minmax.transform(x_train)
x_valid_new2 = minmax.transform(x_valid)
x_test_new2 = minmax.transform(x_test)

model = LinearRegression()
model.fit(x_train_new2, y_train)

y_valid_pred2 = model.predict(x_valid_new2)
y_test_pred2 = model.predict(x_test_new2)

print(mean_squared_error(y_valid, y_valid_pred2))

print(model.coef_)
abscoef = abs(model.coef_)
print(abscoef/abscoef.max())

44600692064.98767
[-1299949.29783402   483243.83316135   799555.6332045    -79891.06258461
    45813.61561185   543770.59419951   180678.50083123    59109.63331034
  1434821.53911522   908312.80399398   479315.22148021  -427883.52344607
     1639.78732943   130604.70441674  -376530.26613127]
[0.90600069 0.33679717 0.55725093 0.05568014 0.03192984 0.37898134
 0.12592402 0.04119651 1.         0.63304932 0.33405912 0.29821376
 0.00114285 0.09102505 0.26242307]


In [58]:
x_train_new3 = np.log(x_train + 1)
x_valid_new3 = np.log(x_valid + 1)
x_test_new3 = np.log(x_test + 1)

model = LinearRegression()
model.fit(x_train_new3, y_train)

y_valid_pred3 = model.predict(x_valid_new3)
y_test_pred3 = model.predict(x_test_new3)
print(mean_squared_error(y_valid, y_valid_pred3))

print(model.coef_)
abscoef = abs(model.coef_)
print(abscoef/abscoef.max())

51841036974.26161
[-1.41034747e+05  1.69180504e+05 -7.86147361e+04 -6.48169348e+03
  2.84097815e+03  9.04932796e+05  1.17832911e+05  6.87265669e+04
  1.17516296e+06  3.05317691e+05  1.93383459e+04 -7.30527205e+06
  2.04508484e+03  1.10049797e+05 -1.30378800e+04]
[1.93058857e-02 2.31586863e-02 1.07613701e-02 8.87262436e-04
 3.88894230e-04 1.23873935e-01 1.61298457e-02 9.40780390e-03
 1.60865051e-01 4.17941576e-02 2.64717669e-03 1.00000000e+00
 2.79946431e-04 1.50644351e-02 1.78472203e-03]


## 3. Отбор признаков

1. Воспользуйтесь методом `sklearn.feature_selection.RFE` для отбора 5 наиболее важных признаков. 
- сравните как меняется результат для 3 подходов к стандартизации признаков и для случая без нормировки
- в рамках каждого подхода сравните отобранные 10 признаков с 10 признаками имеющими наибольшую важность для базовой модели (обученной на всех данных). Подготовьте таблицу важности признаков, в которой каждый столбец соответствует какому-то методу нормировки данных. Сравните какие признаки оказываются важными в разных сценариях. Прокомментируйте полученный результат.

|LR|LR-standartized|LR-normalized|LR-log|LR-standartized-RFE|LR-normalized-RFE|LR-log-RFE|

2. Оцените качество каждой из 7 обученных моделей на тестовой части данных `kc_house_data_test.csv` не забудьте применить соответствующие преобразования (стандартизация и пр). Совпадают ли лучшая модель на валидации и на тесте?

In [59]:
model.fit(x_train_new1, y_train)

rfe = RFE(model, n_features_to_select=5)
rfe.fit(x_train_new1, y_train)

y_pred = rfe.predict(x_valid_new1)
y_test_pred1_rfe = rfe.predict(x_test_new1)

print(mean_squared_error(y_valid, y_pred))
print(rfe.support_)

coef = abs(rfe.estimator_.coef_)
print(coef/coef.max())

47761912049.86946
[False False  True False False  True False False  True  True False  True
 False False False]
[1.         0.36073645 0.98770145 0.05323663 0.6126772 ]


In [60]:
model.fit(x_train_new2, y_train)

rfe = RFE(model, n_features_to_select = 5)
rfe.fit(x_train_new2, y_train)

y_pred = rfe.predict(x_valid_new2)
y_test_pred2_rfe = rfe.predict(x_test_new2)

print(mean_squared_error(y_valid, y_pred))
print(rfe.support_)

abscoef = abs(rfe.estimator_.coef_)
print(abscoef/abscoef.max())

59241761832.89005
[ True False  True False False False False False  True  True  True False
 False False False]
[1.         0.75724767 0.89798195 0.74429017 0.67336209]


In [61]:
model.fit(x_train_new3, y_train)

rfe = RFE(model, n_features_to_select = 5)
rfe.fit(x_train_new3, y_train)

y_pred = rfe.predict(x_valid_new3)
y_test_pred3_rfe = rfe.predict(x_test_new3)

print(mean_squared_error(y_valid, y_pred))
print(rfe.support_)

abscoef = abs(rfe.estimator_.coef_)
print(abscoef/abscoef.max())

56025767959.23278
[False  True  True False False  True False False  True False False  True
 False False False]
[0.02264716 0.02489951 0.14544684 0.18054157 1.        ]


In [62]:
model.fit(x_train, y_train)

rfe = RFE(model, n_features_to_select = 5)
rfe.fit(x_train, y_train)

y_pred = rfe.predict(x_valid)
print(mean_squared_error(y_valid, y_pred))
print(rfe.support_)

abscoef = abs(rfe.estimator_.coef_)
print(abscoef/abscoef.max())

59826960746.653175
[False  True False False False  True  True  True  True False False False
 False False False]
[0.1320748  1.         0.15665896 0.12300473 0.29458229]


In [63]:
print(mean_squared_error(y_test, y_test_pred))
print(mean_squared_error(y_test, y_test_pred1))
print(mean_squared_error(y_test, y_test_pred2))
print(mean_squared_error(y_test, y_test_pred3))
print(mean_squared_error(y_test, y_test_pred1_rfe))
print(mean_squared_error(y_test, y_test_pred2_rfe))
print(mean_squared_error(y_test, y_test_pred3_rfe))

46611588405.96534
46611588405.96165
46611588405.96165
54179215275.6223
48419454301.99658
60634950053.95871
57209745331.27951


## Подготовьте репорт о выполненном домашнем задании

Подготовьте текстовый репорт о выполенном домашнем задании. 

Вы можете вставлять в текст таблицы и/или графики на ваше усмотрение, текст репорта должен быть в двухколоночном формате не более 2 страниц 12 кеглем Times New Roman (крайне желательно 1 страница), см. шаблон https://docs.google.com/document/d/1YqnO0OFbTR7XgzmdWryQF22J1kl9Fg7hSlNbqa6Jbiw/edit?usp=sharing

Подготовленный репорт и jupyter notebook c вашим решением необходимо отправить на почту kurmukovai@gmail.com, с темой письма [iitp-intro-ds-2023-ha2-Surname], например [iitp-intro-ds-2023-ha2-Kurmukov] до 23:59:59 МСК 04.03.2023.